# Benchmarking

We want to test the behavior of ProcessOptimizer on different model systems with
different settings. First, we create a list of dicts, where each dict contains the
settings to use.

In [ ]:
from itertools import product
from typing import Union

from ProcessOptimizer.model_systems import get_model_system
from ProcessOptimizer import Optimizer

MODEL_SYSTEM_NAMES = [
    "branin_hoo",
    "hart3",
    "hart6",
]
EXPERIMENT_BUDGET = 100 # How many evaluations we can do in total per optimization
NUM_REPLICATIONS = 20 # How many times to perform each optimization
NOISE_LEVELS = [0.0, 0.2, 1.0, 5.0, 10.0] # What to multiply the noise of the modelsystem by
TARGET_LEVEL = [0.01, 0.1] # How close to the true minimum we want to get
N_INITIAL_POINTS = [4, 10]

seed = 0 # Seed for "randomly" making noise, ensures reproducibility
tests : list[dict[str, Union[str, float, int]]] = [] # Consider making a "test" dataclass for better typing
for model_system_name, relative_noise_level, n_initial_points, target_level in product(
    MODEL_SYSTEM_NAMES,
    NOISE_LEVELS,
    N_INITIAL_POINTS,
    TARGET_LEVEL
):
    for _ in range(NUM_REPLICATIONS): # Adding NUM_REPLICATIONS tests for each combination
        seed += 1 # Each test should have a different seed
        test = {
            "model_system_name": model_system_name,
            "noise_level": relative_noise_level,
            "n_initial_points": n_initial_points,
            "target_level": target_level,
            "experiment_budget": EXPERIMENT_BUDGET,
            "seed": seed
        }
        tests.append(test)

Then, we create the function to run the test on each member of the list.

In [ ]:
def run_test_optimization(
        test: dict[str, Union[str, float, int]]
) -> tuple[dict, int, bool]:
    """
    Run an optimization test and return the number of evaluations done and whether the
    optimization was successful

    Parameters
    ----------
    test : dict
        A dictionary containing the test parameters.

    Returns
    -------
    tuple
        A tuple containing the input test (for reference), the number of evaluations done
        and whether the optimization was successful.
    """
    model_system = get_model_system(test["model_system_name"], seed = test["seed"])
    model_system.noise_level = model_system.noise_level*test["noise_level"]
    range = model_system.true_max - model_system.true_min
    target = model_system.true_min + test["target_level"] * range
    optimizer = Optimizer(
        dimensions=test["model_system"].space,
        n_initial_points=test["n_initial_points"]
    )
    for _ in range(test["experiment_budget"]):
        finished = False
        x = optimizer.ask()
        y = model_system(x)
        if y < target:
            # We have found a point that is close enough to the true minimum
            # There should be some more logic here, to check if the point is acutally
            # good enough, or whether it was just luck.
            finished = True
            break
        optimizer.tell(x, y)
        return (test, len(optimizer.Xi), finished)

We can then run the tests in parallel.

In [ ]:
from multiprocessing import Pool

p = Pool(5)
result = p.map(run_test_optimization, tests)
p.close()